# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [61]:
# Importing Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [62]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'


for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))
    

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [63]:
# initiating an empty list of rows that will be generated from each file

full_data_rows_list=[]

for f in file_path_list:
    with open(f,'r',encoding='utf-8',newline='') as csvfile:
        csvreader=csv.reader(csvfile)
        next(csvreader)
        for line in csvreader:
            full_data_rows_list.append(line)

In [64]:
print(len(full_data_rows_list))

8056


In [65]:
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [66]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of the project. 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [67]:
import cassandra

In [68]:
# making a connection to the Cluster
# as we are running the cluster locally, hence using '127.0.0.1'

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session=cluster.connect()
except Exception as e:
    print("Error in establishing the connection")
    print(e)


#### Create Keyspace

In [69]:
# Key space is just like a database in relational database 
# creating a keyspace

try:
    session.execute("""
    create keyspace if not exists project_cassandra
    with replication =
    {'class':'SimpleStrategy','replication_factor':1 }
    """)
    
# we keep the replication factor = 1 as we are running the code locally

except Exception as e:
    print("Error in creating the KeySpace")
    print(e)
    

#### Set Keyspace

In [70]:
# Setting KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('project_cassandra')
except Exception as e:
    print("error in setting the keyspace")
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries wewant to run.

## We will be creating queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In order to perform this query we need information about the sessions and the songs listened to, so the table that will be build will be composed of the following columns: sessionId, itemInSession, artist, song, and length. The PRIMARY KEY is composed by sessionId as partition key and itemInSession as the clustering
column. This allows to have a unique key that responds to the query requirements.

In [71]:
# creating table session_song_history
query= "create table if not exists song_details "


query=query+"(sessionId int, itemInSession int ,artist text ,song text ,length double, primary key (sessionId,itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print("error in creating the table")
    print(e)

In [72]:

file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # Skipping the header
    for line in csvreader:
        query = "INSERT INTO song_details (sessionId, itemInSession, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))


In [73]:
q="select artist, song, length from song_details where sessionId=338 and itemInSession=4 "
try:
    rows=session.execute(q)
except Exception as e:
    print("error in query")
    print(e)


x = PrettyTable()
x.field_names = ["Artist", "Song", "Length"]
for row in rows:
    x.add_row([row.artist, row.song, row.length])

print(x)

+-----------+---------------------------------+----------+
|   Artist  |               Song              |  Length  |
+-----------+---------------------------------+----------+
| Faithless | Music Matters (Mark Knight Dub) | 495.3073 |
+-----------+---------------------------------+----------+


### Query2 : Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In order to perform this query we need information about the userId and the sessionId of users, so the table that will be build will be composed of the following columns: userID, sessionId, itemInSession, artist, song, firstName, lastName. 
The PRIMARY KEY is a composite primary key by (userId, sessionId) as partition key and itemInSession as the clustering
column. This allows to have a unique key that responds to the query requirements.

In [74]:
# creating table artist_details

query= "create table if not exists artist_details"
query=query+"(userId int ,sessionId int, itemInSession int ,artist text ,song text ,firstName text, lastName text, primary key ((userId,sessionId),itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print("error in creating the table")
    print(e)




                    

In [75]:
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # Skipping the header
    for line in csvreader:
        query = "INSERT INTO artist_details (userId, sessionId, itemInSession, artist, song, firstName, lastName) "
        query += "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))


In [76]:
q="select artist, song, firstname, lastname from artist_details where userId=10 and sessionId=182 "
try:
    rows=session.execute(q)
except Exception as e:
    print("error in query")
    print(e)

    
    
x = PrettyTable()
x.field_names = ["Artist", "Song", "First Name", 'Last Name']
for row in rows:
    x.add_row([row.artist, row.song, row.firstname,row.lastname])

print(x)

+-------------------+------------------------------------------------------+------------+-----------+
|       Artist      |                         Song                         | First Name | Last Name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


### Query3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own

In order to perform this query we need information about the song and userId so that we can uniquely represent the songs, 
so the table that will be build will be composed of the following columns: song, firstName, lastName and user_id 
The PRIMARY KEY is a composite primary key by (song, user_id) as partition key.
column. This allows to have a unique key that responds to the query requirements.

In [77]:
# creating table user_details


query= "create table if not exists user_details"
query=query+"(song text ,firstName text, lastName text, user_id int, primary key (song,user_id))"
try:
    session.execute(query)
except Exception as e:
    print("error in creating the table")
    print(e)




                    

                    

In [78]:
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # Skipping the header
    for line in csvreader:
        query = "INSERT INTO user_details (song, firstName, lastName, user_id) "
        query += "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))


In [79]:
q="select firstname, lastname from user_details where song='All Hands Against His Own' "
try:
    rows=session.execute(q)
except Exception as e:
    print("error in query")
    print(e)

    
x = PrettyTable()
x.field_names = ["First Name", "Last Name"]
for row in rows:
    x.add_row([row.firstname, row.lastname])

print(x)

+------------+-----------+
| First Name | Last Name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Drop the tables before closing out the sessions

In [81]:
for t in ["song_details", "artist_details", "user_details"]:
    query = f"DROP TABLE {t}"
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [82]:
session.shutdown()
cluster.shutdown()